# Defining neural networks with Keras

## Sequential API

![image-14](image-14.png)

- A good way to construct this model in Keras is to use the sequential API. 
- This API is simpler and makes strong assumptions about how you will construct your model. 
- It assumes that you have an input layer, some number of hidden layers, and an output layer. 
- All of these layers are ordered one after the other in a sequence.

In [1]:
from tensorflow import keras

# Define a Keras sequential model
model = keras.Sequential()

# Define the first dense layer
model.add(keras.layers.Dense(16, activation='relu', input_shape=(784,)))

# Define the second dense layer
model.add(keras.layers.Dense(8, activation='relu'))

# Define the output layer
model.add(keras.layers.Dense(4, activation='softmax'))

# Print the model architecture
print(model.summary())

2023-05-24 02:02:17.773172: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-24 02:02:17.773196: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                12560     
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 4)                 36        
                                                                 
Total params: 12,732
Trainable params: 12,732
Non-trainable params: 0
_________________________________________________________________
None


2023-05-24 02:02:19.270771: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-24 02:02:19.270792: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-24 02:02:19.270808: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (9333b831-c81f-4fcc-aebe-67a924348f1e): /proc/driver/nvidia/version does not exist
2023-05-24 02:02:19.270999: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Define a Keras sequential model
model = keras.Sequential()

# Define the first dense layer
model.add(keras.layers.Dense(16, activation = 'sigmoid', input_shape=(784,)))

# Apply dropout to the first layer's output
model.add(keras.layers.Dropout(0.25))

# Define the output layer
model.add(keras.layers.Dense(4, activation='softmax'))

# Compile the model
model.compile('adam', loss='categorical_crossentropy')

# Print a model summary
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 16)                12560     
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_4 (Dense)             (None, 4)                 68        
                                                                 
Total params: 12,628
Trainable params: 12,628
Non-trainable params: 0
_________________________________________________________________
None


## Functional API

![image-15](image-15.png)

-  If you want to train two models jointly to predict the same target, functional API is for that.

In [3]:
import pandas as pd
import tensorflow as tf

slang_func = pd.read_csv('datasets/slmnist.csv', header=None)
slang_func.shape

(2000, 785)

In [4]:
import numpy as np

# Cretaing 2 inputs 
inputs_1_features = np.array(slang_func.drop(0,axis=1), dtype=np.float32)
inputs_2_features = np.array(slang_func.drop(0,axis=1), dtype=np.float32)

targets = np.array(pd.get_dummies(slang_func[0]), dtype=np.float32)

In [5]:
# Creating architecture for each input
inputs1 = tf.keras.Input(shape=(784,))
hidden_1_1 = tf.keras.layers.Dense(16, activation='relu')(inputs1)
hidden_2_1 = tf.keras.layers.Dense(8, activation='relu')(hidden_1_1)
outputs_1 = tf.keras.layers.Dense(4, activation='softmax')(hidden_2_1)

inputs2 = tf.keras.Input(shape=(784,))
hidden_1_2 = tf.keras.layers.Dense(16, activation='sigmoid')(inputs2)
hidden_2_2 = tf.keras.layers.Dense(8, activation='relu')(hidden_1_2)
outputs_2 = tf.keras.layers.Dense(4, activation='softmax')(hidden_2_2)

In [6]:
# Merge the outputs 
merged = tf.keras.layers.add([outputs_1,outputs_2])

In [7]:
# Create functional model
model = tf.keras.models.Model(inputs=[inputs1, inputs2], outputs=merged)

# Summary of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 784)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 784)]        0           []                               
                                                                                                  
 dense_5 (Dense)                (None, 16)           12560       ['input_1[0][0]']                
                                                                                                  
 dense_8 (Dense)                (None, 16)           12560       ['input_2[0][0]']                
                                                                                              

In [8]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Fit the model
model.fit([inputs_1_features, inputs_2_features], targets, epochs=10, validation_split=0.2)

Epoch 1/10
50/50 [==============================] - 1s 4ms/step - loss: 1.6873 - accuracy: 0.2625 - val_loss: 1.7271 - val_accuracy: 0.2425
Epoch 2/10
50/50 [==============================] - 0s 3ms/step - loss: 1.6514 - accuracy: 0.2519 - val_loss: 1.6417 - val_accuracy: 0.2425
Epoch 3/10
50/50 [==============================] - 0s 2ms/step - loss: 1.5993 - accuracy: 0.2519 - val_loss: 1.6068 - val_accuracy: 0.2425
Epoch 4/10
50/50 [==============================] - 0s 2ms/step - loss: 1.5721 - accuracy: 0.2519 - val_loss: 1.5809 - val_accuracy: 0.2425
Epoch 5/10
50/50 [==============================] - 0s 2ms/step - loss: 1.5534 - accuracy: 0.2519 - val_loss: 1.5640 - val_accuracy: 0.2425
Epoch 6/10
50/50 [==============================] - 0s 2ms/step - loss: 1.5425 - accuracy: 0.2519 - val_loss: 1.5547 - val_accuracy: 0.2425
Epoch 7/10
50/50 [==============================] - 0s 2ms/step - loss: 1.5358 - accuracy: 0.2519 - val_loss: 1.5484 - val_accuracy: 0.2425
Epoch 8/10
50/50 [==

# Training and validation with Keras

1. Build the model with layers and activation functions
2. Compile with optimizers and loss 
3. Fit the model to training data (Train the model)
4. Evaluate on training and testing sets


## The fit() operation
- Required arguments:
    - `features`
    - `labels`

- Many optional arguments
    - `batch_size` : The number of examples in each batch is the batch size, which is 32 by default. 
    - `epochs` : The number of times you train on the full set of batches is called the number of epochs.
    - `validation_split` :It divides the dataset into two parts. The first part is the train set and the second part is the validation set.

In [10]:
import pandas as pd
slang = pd.read_csv('datasets/slmnist.csv', header=None)
slang.shape

(2000, 785)

In [11]:
import numpy as np

# Define features and labels
sign_language_features = np.array(slang.drop(0,axis=1), dtype=np.float32)
sign_language_labels = np.array(pd.get_dummies(slang[0]), dtype=np.float32)

## Training with Keras

In [12]:
# Define a sequential model
model = keras.Sequential()

# Define a hidden layer
model.add(keras.layers.Dense(16, activation='relu', input_shape=(784,)))

# Define the output layer
model.add(keras.layers.Dense(4, activation='softmax'))

# Compile the model
model.compile('SGD', loss='categorical_crossentropy')

# Complete the fitting operation
model.fit(sign_language_features, sign_language_labels, epochs=5)

Epoch 1/5
63/63 [==============================] - 0s 798us/step - loss: 12898.8633
Epoch 2/5
63/63 [==============================] - 0s 784us/step - loss: 1.3864
Epoch 3/5
63/63 [==============================] - 0s 849us/step - loss: 1.3864
Epoch 4/5
63/63 [==============================] - 0s 830us/step - loss: 1.3864
Epoch 5/5
63/63 [==============================] - 0s 863us/step - loss: 1.3864


## Metrics and validation with Keras

In [13]:
# Define features and labels
sign_language_features = np.array(slang.drop(0,axis=1), dtype=np.float32)
sign_language_labels = np.array(pd.get_dummies(slang[0]), dtype=np.float32)

In [14]:
# Define sequential model
model = keras.Sequential()

# Define the first layer
model.add(keras.layers.Dense(32, activation='sigmoid', input_shape=(784,)))

# Add activation function to classifier
model.add(keras.layers.Dense(4, activation='softmax'))

# Set the optimizer, loss function, and metrics
model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Add the number of epochs and the validation split
model.fit(sign_language_features, sign_language_labels, epochs=10, validation_split=0.1)

Epoch 1/10
57/57 [==============================] - 0s 3ms/step - loss: 1.5197 - accuracy: 0.2544 - val_loss: 1.4256 - val_accuracy: 0.2150
Epoch 2/10
57/57 [==============================] - 0s 1ms/step - loss: 1.3923 - accuracy: 0.2733 - val_loss: 1.3865 - val_accuracy: 0.2550
Epoch 3/10
57/57 [==============================] - 0s 1ms/step - loss: 1.3736 - accuracy: 0.3089 - val_loss: 1.3957 - val_accuracy: 0.2150
Epoch 4/10
57/57 [==============================] - 0s 1ms/step - loss: 1.3856 - accuracy: 0.2611 - val_loss: 1.3853 - val_accuracy: 0.2150
Epoch 5/10
57/57 [==============================] - 0s 1ms/step - loss: 1.3649 - accuracy: 0.3522 - val_loss: 1.3888 - val_accuracy: 0.2200
Epoch 6/10
57/57 [==============================] - 0s 1ms/step - loss: 1.3520 - accuracy: 0.3811 - val_loss: 1.3895 - val_accuracy: 0.2200
Epoch 7/10
57/57 [==============================] - 0s 1ms/step - loss: 1.3304 - accuracy: 0.4411 - val_loss: 1.3124 - val_accuracy: 0.4700
Epoch 8/10
57/57 [==

## Overfitting detection

In [15]:
# Define features and labels
sign_language_features = np.array(slang.drop(0,axis=1), dtype=np.float32)
sign_language_labels = np.array(pd.get_dummies(slang[0]), dtype=np.float32)

In [16]:
# Define sequential model
model = keras.Sequential()

# Define the first layer
model.add(keras.layers.Dense(1024, activation='relu', input_shape=(784,)))

# Add activation function to classifier
model.add(keras.layers.Dense(4, activation='softmax'))

# Finish the model compilation
model.compile(optimizer=keras.optimizers.Adam(lr=0.001), 
              loss='categorical_crossentropy', metrics=['accuracy'])

# Complete the model fit operation
model.fit(sign_language_features, sign_language_labels, epochs=50, validation_split=0.5)

Epoch 1/50
32/32 [==============================] - 0s 8ms/step - loss: 382.3554 - accuracy: 0.4120 - val_loss: 39.1774 - val_accuracy: 0.7080
Epoch 2/50
32/32 [==============================] - 0s 7ms/step - loss: 12.9987 - accuracy: 0.7690 - val_loss: 16.0303 - val_accuracy: 0.7020
Epoch 3/50
32/32 [==============================] - 0s 8ms/step - loss: 9.9020 - accuracy: 0.8250 - val_loss: 9.4544 - val_accuracy: 0.7860
Epoch 4/50
32/32 [==============================] - 0s 6ms/step - loss: 2.5709 - accuracy: 0.8990 - val_loss: 0.9110 - val_accuracy: 0.9630
Epoch 5/50
32/32 [==============================] - 0s 6ms/step - loss: 3.4397 - accuracy: 0.9010 - val_loss: 18.2838 - val_accuracy: 0.7220
Epoch 6/50
32/32 [==============================] - 0s 6ms/step - loss: 3.0256 - accuracy: 0.9290 - val_loss: 0.3639 - val_accuracy: 0.9810
Epoch 7/50
32/32 [==============================] - 0s 7ms/step - loss: 0.1987 - accuracy: 0.9890 - val_loss: 2.6219 - val_accuracy: 0.9280
Epoch 8/50
32/

**Notice** that the validation loss, val_loss, was substantially higher than the training loss, loss. Furthermore, if val_loss started to increase before the training process was terminated, then we may have overfitted. When this happens, you will want to try decreasing the number of epochs.

## Evaluating models

In [28]:
# Define features and labels
sign_language_features = np.array(slang.drop(0,axis=1), dtype=np.float32)
sign_language_labels = np.array(pd.get_dummies(slang[0]), dtype=np.float32)

In [29]:
# Split for evaluation
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sign_language_features, sign_language_labels,
                                                   test_size=0.3,
                                                   random_state=42)

In [30]:
import tensorflow as tf

# Create sequential model 
model = tf.keras.Sequential()

# Design first layer
model.add(tf.keras.layers.Dense(16, activation='relu', input_shape=(784,)))

# Design second layer
model.add(tf.keras.layers.Dense(8, activation='relu'))

# Design output layer
model.add(tf.keras.layers.Dense(4, activation='softmax'))

# Summary of the model
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 16)                12560     
                                                                 
 dense_21 (Dense)            (None, 8)                 136       
                                                                 
 dense_22 (Dense)            (None, 4)                 36        
                                                                 
Total params: 12,732
Trainable params: 12,732
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compiling with optimizer and loss function
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

# Fit the model
model.fit(X_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
35/35 [==============================] - 0s 4ms/step - loss: 105.2926 - accuracy: 0.3482 - val_loss: 10.7929 - val_accuracy: 0.2893
Epoch 2/10
35/35 [==============================] - 0s 2ms/step - loss: 2.6511 - accuracy: 0.3705 - val_loss: 1.3604 - val_accuracy: 0.2857
Epoch 3/10
35/35 [==============================] - 0s 2ms/step - loss: 1.2184 - accuracy: 0.3518 - val_loss: 1.0279 - val_accuracy: 0.4286
Epoch 4/10
35/35 [==============================] - 0s 2ms/step - loss: 1.0356 - accuracy: 0.5116 - val_loss: 0.9882 - val_accuracy: 0.5393
Epoch 5/10
35/35 [==============================] - 0s 1ms/step - loss: 0.9683 - accuracy: 0.5804 - val_loss: 0.9273 - val_accuracy: 0.6071
Epoch 6/10
35/35 [==============================] - 0s 1ms/step - loss: 0.9235 - accuracy: 0.6036 - val_loss: 0.8874 - val_accuracy: 0.5750
Epoch 7/10
35/35 [==============================] - 0s 2ms/step - loss: 0.9171 - accuracy: 0.5562 - val_loss: 0.8531 - val_accuracy: 0.5857
Epoch 8/10
35/35 

In [32]:
# Evaluate the train 
train = model.evaluate(X_train, y_train)

# Evaluate the test
test = model.evaluate(X_test, y_test)


19/19 [==============================] - 0s 771us/step - loss: 0.7368 - accuracy: 0.6850


In [33]:
print("Train: Loss = ",train[0], ", Accuracy = ", train[1])
print("Test: Loss = ",test[0], ", Accuracy = ", test[1])

Train: Loss =  0.7640196681022644 , Accuracy =  0.6485714316368103
Test: Loss =  0.7368065714836121 , Accuracy =  0.6850000023841858


# Training models with Estimator API

![image-16](image-16.png)

- High level submodule
- Less flexible
- Enforces best practices
- Faster deployment
- Many premade models

## Model specification and training

1. **Define feature columns** : specify the shape and type of your data.
2. **Load and transform data** : load and transform your data within a function. The output of this function will be a dictionary object of features and your labels.
3. **Define an estimator** : use premade estimators or define custom estimators with different architectures.
4. **Apply train operation** : train the model you defined.

In [23]:
housing = pd.read_csv('datasets/kc_house_data.csv')
housing.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [24]:
# Define feature columns for bedrooms and bathrooms
bedrooms = tf.feature_column.numeric_column("bedrooms")
bathrooms = tf.feature_column.numeric_column('bathrooms')

# Define the list of feature columns
feature_list = [bedrooms, bathrooms]

# Load and transform data
def input_fn():
	# Define the labels
	labels = np.array(housing['price'])
	# Define the features
	features = {'bedrooms':np.array(housing['bedrooms']), 
                'bathrooms':np.array(housing['bathrooms'])}
	return features, labels

In [25]:
# Define the deep neural network regressor model with 2 nodes in both the first and second hidden layers
model = tf.estimator.DNNRegressor(feature_columns=feature_list, hidden_units=[2,2])
model.train(input_fn, steps=10) # <-- 10 training step

2023-05-24 02:02:35.682055: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2023-05-24 02:02:35.685204: W tensorflow/core/common_runtime/forward_type_inference.cc:231] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT64
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}

	while inferring type of node 'dnn/zero_fraction/cond/output/_18'


In [26]:
model.evaluate(input_fn, steps=10)

{'average_loss': 426474570000.0,
 'label/mean': 540088.25,
 'loss': 426474540000.0,
 'prediction/mean': -2.7017937,
 'global_step': 10}